In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
train_data = pd.read_csv('../input/disease-prediction-using-machine-learning/Training.csv')
test_data =  pd.read_csv('../input/disease-prediction-using-machine-learning/Testing.csv')

In [3]:
x = train_data.iloc[:,:-2]
y = train_data.loc[:,'prognosis']

enc = LabelEncoder(); enc.fit(y)
train_data['categorical_label'] = enc.transform(y)
y = train_data['categorical_label']

# Exploratory Data Analysis

In [4]:
train_data.head(5)

In [5]:
del train_data["Unnamed: 133"]

In [6]:
train_data.describe()

In [7]:
train_data.info()

In [8]:
train_data.isnull().sum()

In [9]:
plt.figure(figsize=(20,5))
plt.xticks(rotation=45)
sns.countplot(train_data["prognosis"])
print(train_data.prognosis.value_counts())

In [33]:
for i in x.columns:
    if x[i].sum() == 0:
        x.drop(i,axis=1)

In [11]:
x.corr()

In [12]:
def correlated(df):
    correl = set()
    c = df.corr()
    for i in range(len(c.columns)):
        for j in range(i):
            if c.iloc[i,j]>0.8:
                correl.add(c.columns[i])
    return correl
corr_col = correlated(x)
len(corr_col)

In [13]:
x = x.drop(corr_col,axis=1)

In [14]:
x.columns

## Analysing diseases which have common symptoms

In [15]:
df = train_data
for i in x.columns:
 if i == "spotting_ urination":
     smptm = df.loc[df['spotting_ urination']==1].groupby('prognosis')['spotting_ urination'].sum()
 else:
     exec("smptm = df.loc[df."+i+"==1].groupby('prognosis')."+i+".sum()")
 smptm = pd.DataFrame(smptm)
 smptm['Percent'] = (smptm[i]/smptm[i].sum())*100
 print(smptm)
 plt.figure(figsize=(5,5))
 plt.xticks(rotation=45)
 plt.title(i)
 sns.barplot(x=smptm.index,y=smptm[i])
 plt.ylabel("count")
 plt.show()

In [ ]:
#train_data.hist()

In [16]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=2)
x_test.shape

## Decision Tree Classifier

In [17]:
from sklearn.tree import DecisionTreeClassifier
scores = []
for i in range(1,50):
 dtc = DecisionTreeClassifier(max_depth=i).fit(x,y)
 y_pred_dtc = dtc.predict(x_test)
 scores.append(dtc.score(x_test,y_test))

In [18]:
y_pred_dtc

In [19]:
fine_depth = scores.index(max(scores))
scores[fine_depth]

### Confusion Matrix

In [20]:
from sklearn.metrics import confusion_matrix
conf_mx = confusion_matrix(y_test,y_pred_dtc)
plt.matshow(conf_mx, cmap=plt.cm.gray)
plt.show()

### Precision for each class

In [21]:
from sklearn.metrics import precision_score
prec = precision_score(y_test,y_pred_dtc,average=None)
print("Precision for each class: ",prec)

## Random Forest Classifier


In [22]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(x,y)
y_pred_rfc = rfc.predict(x_test)
y_pred_rfc

In [23]:
# Accuracy score
print("Training set accuracy: ",rfc.score(x,y))
print("Test set accuracy: ",rfc.score(x_test,y_test))

### Confusion Matrix

In [24]:
from sklearn.metrics import confusion_matrix
conf_mx = confusion_matrix(y_test,y_pred_rfc)
plt.matshow(conf_mx, cmap=plt.cm.gray)
plt.show()

### Precision Score for each class

In [25]:
from sklearn.metrics import precision_score
prec = precision_score(y_test,y_pred_rfc,average=None)
print("Precision for each class: ",prec)

## Naive Bayes Classifier

In [26]:
from sklearn.naive_bayes import GaussianNB
nbc = GaussianNB().fit(x,y)
y_pred_nbc = nbc.predict(x_test)
y_pred_nbc

In [27]:
#Accuracy Score
print("Training set accuracy: ",nbc.score(x,y))
print("Test set accuracy: ",nbc.score(x_test,y_test))

### Confusion Matrix

In [28]:
from sklearn.metrics import confusion_matrix
conf_mx = confusion_matrix(y_test,y_pred_nbc)
plt.matshow(conf_mx, cmap=plt.cm.gray)
plt.show()

### Precision Score for each class

In [29]:
from sklearn.metrics import precision_score
prec = precision_score(y_test,y_pred_nbc,average=None)
print("Precision for each class: ",prec)

## MLPClassifier

In [30]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(solver='adam', alpha=1e-5, max_iter=500, hidden_layer_sizes=(15, 7), random_state=10)
mlp.fit(x,y)
y_pred_mlp = mlp.predict(x_test)
y_pred_mlp

In [31]:
#Accuracy Score
print("Training set accuracy: ",nbc.score(x,y))
print("Test set accuracy: ",mlp.score(x_test,y_test))

## Confusion Matrix

In [32]:
from sklearn.metrics import confusion_matrix
conf_mx = confusion_matrix(y_test,y_pred_mlp)
plt.matshow(conf_mx, cmap=plt.cm.gray)
plt.show()